# Processing Pasadena (Texas) jail logs: 8/18 - 5/21

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt
import os

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read the raw data

The department provided the data in a [messy PDF format](data/raw/Restraint_Chair_Data.pdf). That limits this initial analysis in two ways: 1) some of the pages are scanned as images, rendering them flat (and impervious to PDF-to-text parsing software); and 2) the redactions in the PDF's narrative field make it impossible to extract in a machine readable format. We can talk about strategies for getting around this. It should also be noted that there are roughly 100 missing records because of the flat pages issue. Nevertheless, I was able to extract roughly 500 records from the PDF into a dataframe for analysis. 

In [4]:
# src = pd.read_csv(
#     "data/raw/chair-slices/tabula-chair-columns_0-6.csv",
#     parse_dates=["date"],
#     dtype={"id": "str"},
# )

In [5]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQFl6SwQ0V_c4hU0R8gpbnUAOQJyfezGdoFpzQabLVYmCpH8wD-oSmU9fCs3qKHXftaNdPp72I3t1sn/pub?output=csv",
    header=None,
    names=[
        "id",
        "date",
        "time",
        "shift",
        "reporting_personnel",
        "number_assisting",
        "location",
        "narrative",
    ],
    parse_dates=["date"],
    dtype={"id": "str", "time": str},
)

In [6]:
src.sort_values("date", ascending=False).head()

,id,date,time,shift,reporting_personnel,number_assisting,location,narrative
606,202100126,2021-05-05,2240,NITE,"KENNEDY, J",3.0,DWI \nROOM,"Prisoner came into the jail intoxicated and was restrained in the chair due to the male needing a blood draw. [05/06/2021 01:08, JEKENNEDY, , PPD] Prisoner arrived to the DWI room and at removal of handcuffs by Ofc Green the inmate threw himself on to \nPAY \nthe ground stating he needs medical. Inmate was observed to have dry blood on his right hand and stated he was in pain and that he was diabetic. Inmate was then checked and further talked to in spanish by PSO Villegas. PSO Villegas advised that he was in pain due to his hand hurting and was given instructions to sit up and to allow for further search of property. Prisoner refused and continued to lay on ground. Officer Green advised that he was going to be a Magistrate Blood Draw and that restraint chair would be required for inmate.Inmate was restrained without further incident. [05/06/2021 01:13, JEKENNEDY, , PPD] \nPAY"
605,202100124,2021-04-30,100,NITE,"VILLEGAS, L",4.0,SALLY \nPORT,"Inmate was restrained in Sally Port at arrival due to being belligerent at arrest location. No further incident occurred. [04/30/2021 03:55, LVILLEGAS, , PPD] \nPA"
604,202100121,2021-04-27,2335,NITE,"VILLEGAS, L",2.0,BOOKING AREA,"INMATE ARRIVED BELLIGERENT AND STARTED PULLING FROM OFFICER HERNANDEZ AND MYSELF. PRISONER WAS INFORMED TO COOPERATE AND SHE CONTINUED TO PULL AND TRY TO STOP FROM ENTERING THE FACILITY. [04/28/2021 01:52, LVILLEGAS, , PPD] \nPA"
603,202100116,2021-04-25,1000,DAYS,"MARROQUIN, J",3.0,BOOKING AREA,"Officer Powell informed Medina,Christopher Booking No 258324 that he would be getting his blood drawn due to his \nDOB\nDWI charge. Medina stated he would not comply. Once Acadian arrived to the jail, Medina was placed in the restraint chair by Officer Jones, Officer Doskocil and myself. There was minimal resistance from Medina during the process. All straps were checked and he was safely secured in the restraint chair. During the blood draw, Medina began to make threats towards the arresting officer claiming he would beat him up once released from the restraint chair. Medina was also verbally abusive towards the medic that drew his blood. Medina remained in the restraint chair for the officer and jail staff safety. PSO Supervisor Davis was notified of the incident. [04/25/2021 13:16, JMARROQUIN, , PPD] \nPA"
602,202100114,2021-04-20,520,NITE,"BETANCOURT, L",3.0,CELL,"At approximately 0520 prisoner Gomez, Lance Giovanni Booking #258265 was behaving erratically and placed in \nDOB\nthe restraint chair after continously hitting the cell door, punching the phone and kicking the toilet on multiple times. Prisoner Gomez refused to stop and became belliegerent after being advised that jail staff would have to place him in a restraint chair for his own safety and to prevent damage to the facility. PSO Supervisor Adame attempted to deescalate the situation but the prisoner again stated that he would not calm down and wanted to be placed in to the restraint chair. Pso Burden, Pso Adame, and myself Pso Betancourt placed the prisoner in to the restraint chair with some resistance but no injuries to jail staff or the inmate. PSO Supervisor Burden was present on scene. [04/20/2021 05:50, LBETANCOURT, , PPD] \nPA"


### Dates

In [7]:
src["date"] = pd.to_datetime(src["date"], format="%m/%d/%Y")
src["date"] = pd.to_datetime(src["date"].dt.strftime("%Y-%m-%d"))
src["year"] = src["date"].dt.year
src["quarter"] = src["date"].dt.quarter
src["day"] = src["date"].dt.day
src["month"] = src["date"].dt.month
src["weekday"] = src["date"].dt.day_name()
src["monthname"] = src["date"].dt.month_name()
src["month_year"] = pd.to_datetime(src["date"]).dt.to_period("M")
src["month_year_full"] = src["date"].apply(lambda x: x.strftime("%B-%Y"))
src["month_year_full"] = pd.to_datetime(src["month_year_full"])

### Times

In [8]:
src["time"] = src["time"].astype(str).str.zfill(4)
src["hour"] = src["time"].str[:2]
src["minute"] = src["time"].str[-2:]
src["time_full"] = pd.to_datetime(
    src["hour"] + ":" + src["minute"], format="%H:%M"
).dt.time
src["hour"] = src["hour"].astype(int)
src["minute"] = src["minute"].astype(int)

In [9]:
src["location"] = src["location"].str.replace("\n", " ", regex=False)

### Make a copy of the dataframe and rearrange the columns

In [10]:
df = src[
    [
        "id",
        "date",
        "time",
        "shift",
        "reporting_personnel",
        "number_assisting",
        "location",
        "year",
        "quarter",
        "day",
        "month",
        "weekday",
        "monthname",
        "month_year",
        "month_year_full",
        "hour",
        "minute",
        "time_full",
        "narrative",
    ]
].copy()

### How many restraint cases do we have now? 

In [11]:
len(df)

607

### At least six pages of restraint cases haven't been parsed

In [12]:
path = "data/raw/restraint_slices_PDF/flat_pdfs_to_parse/"

to_parse = []

for root, dirs, files in os.walk(path):
    for file in files:
        to_parse.append(os.path.join(root, file))

### These are the file names and paths to those pages

In [13]:
to_parse

['data/raw/restraint_slices_PDF/flat_pdfs_to_parse/Restraint_Chair_Data 201900104.pdf',
 'data/raw/restraint_slices_PDF/flat_pdfs_to_parse/Restraint_Chair_Data 202000365.pdf',
 'data/raw/restraint_slices_PDF/flat_pdfs_to_parse/Restraint_Chair_Data 201800003.pdf',
 'data/raw/restraint_slices_PDF/flat_pdfs_to_parse/Restraint_Chair_Data 201900154.pdf',
 'data/raw/restraint_slices_PDF/flat_pdfs_to_parse/Restraint_Chair_Data 202000132.pdf',
 'data/raw/restraint_slices_PDF/flat_pdfs_to_parse/Restraint_Chair_Data 201900086.pdf']

---

### How many restraint cases per year? 

In [14]:
years = df.groupby("year")["id"].size().reset_index(name="count")
years

,year,count
0,2018,157
1,2019,228
2,2020,166
3,2021,56


### How many per month

In [15]:
months = df.groupby(["month", "year"])["id"].size().reset_index(name="count")
months.head()

,month,year,count
0,1,2019,20
1,1,2020,13
2,1,2021,12
3,2,2019,36
4,2,2020,11


In [34]:
chair_months = (
    alt.Chart(months)
    .mark_bar()
    .encode(
        alt.X("month:O", title="Month", axis=alt.Axis(grid=False)),
        y=alt.Y(
            "count:Q",
            title="Count",
            axis=alt.Axis(
                tickCount=6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                title="Restrain incidents per month",
            ),
        ),
        facet=alt.Facet("year:O", columns=4),
    )
    .properties(
        width=300, height=300, title="Chair restraint frequency, by month and year"
    )
)
chair_months.configure_view(strokeOpacity=0)

alt.Chart(...)

In [17]:
df.head()

,id,date,time,shift,reporting_personnel,number_assisting,location,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,narrative
0,201800021,2018-05-22,2100,EVES,"HERRERA, M",NaN,SALLY PORT,2018,2,22,5,Tuesday,May,2018-05,2018-05-01,21,0,21:00:00,"inmate hudson was placed into the restraint chair upon arrival due to officer gassets request. [05/22/2018 21:36, MHERRERA, , \nPA\nPPD]"
1,201800022,2018-05-23,1341,DAYS,"WINGERSON, B",NaN,NaN,2018,2,23,5,Wednesday,May,2018-05,2018-05-01,13,41,13:41:00,"was banging head on wall in 1705 [05/23/2018 13:42, RMCCAIN, , PPD] \nPA"
2,201800027,2018-05-26,1935,EVES,"BETANCOURT, L",5.0,AFIS ROOM,2018,2,26,5,Saturday,May,2018-05,2018-05-01,19,35,19:35:00,"AT around 1935 prisoner Washington, Terrance refused to be moved to his assigned cell after PSO HO, T was done with the ASFIS proccess. PSO HO called over the radio asking for assistance. PSO Thomas, PSO supervisor Rhodes and myself made our way to the AFIS room. The prisoner was again ordered to stand up and walk to his assigned cell and contiued to refuse. PSO Thomas grabbed his arm attempting to escort him when prisoner Terrance began resisiting. The prisoner was taken to the groound and handcuffed. PSO HO and myself helped the prisoner get back on his feet and began walking him into cell 1708 to be placed into the restraint chair. There was no further problems with the prisoner and PSO RHODES was present on scene, [05/26/2018 20:21, LBETANCOURT, , \nPA\nPPD]"
3,201800026,2018-05-26,0400,NITE,"BETANCOURT, L",3.0,AFIS ROOM,2018,2,26,5,Saturday,May,2018-05,2018-05-01,4,0,04:00:00,"While being Afised prisoner Humphrey,Jason Brent requested to be placed into the restraint chair because he \nDOB\nclaimed he would be a danger to jail staff and to himself. The prisoner said he had alot of anger built up insided of him and that he was going to cause alot of damage to the facility if not other wise placed in the restraint chair. I PSO Betancourt along with Pso Weiss, and Supervisor Adame strapped the prisoner to the restraint chair with no problem or resistance and placed the prisoner in isolation cell 1708 for further observation. Supervisor Adame,R was on scene during the event. [05/26/2018 05:08, LBETANCOURT, , PPD] \nPA \n[05/26/2018 05:08, LBETANCOURT, , PPD] [05/26/2018 05:16, LBETANCOURT, , PPD] \nPA PA"
4,201800030,2018-05-27,1800,EVES,"HO, T",4.0,SALLY PORT,2018,2,27,5,Sunday,May,2018-05,2018-05-01,18,0,18:00:00,"While working the jailer position, the inmate Carden,Grant Edwin Booking#239192 was placed in the restraint chair upon arrival at the request of Officer Campbell. Supervisor Degraff was notified and approved. [05/27/2018 18:59, THO, , PPD] [05/27/2018 19:02, PA THO, , PPD] [05/27/2018 19:04, THO, , PPD] He was then escorted to cell 1708 without further incident or injury [05/27/2018 PA PA 19:09, THO, , PPD] PA"


### Which weekdays had the most incidents? 

In [18]:
weekdays = df.groupby("weekday").size().reset_index(name="count")
weekdays.sort_values("count", ascending=False)

,weekday,count
2,Saturday,108
3,Sunday,106
6,Wednesday,89
0,Friday,88
5,Tuesday,86
4,Thursday,72
1,Monday,58


### At which hours were these incidents most common? 

In [19]:
hours = df.groupby("hour").size().reset_index(name="count")
hours.sort_values("count", ascending=False)

,hour,count
2,2,45
1,1,44
3,3,43
0,0,39
21,21,39
17,17,38
4,4,33
20,20,32
15,15,32
14,14,32


### Chart the hours

In [20]:
chair_hours = (
    alt.Chart(hours)
    .mark_bar()
    .encode(
        alt.X("hour:O", title="Hour", axis=alt.Axis(grid=False)),
        y=alt.Y(
            "count:Q",
            title="Count",
            axis=alt.Axis(
                tickCount=6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                title="Restraints per month",
            ),
        ),
    )
    .properties(
        width=600, height=330, title="Chair restraint frequency, by hour in day"
    )
)
chair_hours.configure_view(strokeOpacity=0)

alt.Chart(...)

### Which locations were most common? 

In [21]:
locations = df.groupby("location").size().reset_index(name="count")
locations.sort_values("count", ascending=False)

,location,count
2,CELL,222
1,BOOKING AREA,173
6,SALLY PORT,143
0,AFIS ROOM,28
3,DWI ROOM,9
4,JUVENILE HOLDING AREA,3
5,LAUNDRY ROOM,1


### Who logged restraints the most (i.e., who to call)? 

In [22]:
personnel = df.groupby("reporting_personnel").size().reset_index(name="count")
personnel.sort_values("count", ascending=False).head(10)

,reporting_personnel,count
2,"BETANCOURT, L",88
12,"HO, T",56
18,"PINA, F",47
11,"HERRERA, M",47
15,"MARROQUIN, J",40
29,"WALKER, K",31
24,"THOMAS, R",28
0,"ADAME, R",25
6,"CORTES, A",23
17,"OROZCO, C",22


### Which shifts were most common? 

In [23]:
shifts = df.groupby("shift").size().reset_index(name="count")
shifts.sort_values("count", ascending=False).head(10)

,shift,count
2,NITE,297
1,EVES,255
0,DAYS,55


### Incidents per day in 2020

In [24]:
df2020 = df[df["year"] == 2020].reset_index()

In [25]:
days2020 = df2020.groupby("date").size().reset_index(name="count")
days2020.sort_values("count", ascending=False).head(10)

,date,count
46,2020-05-24,4
52,2020-06-10,4
81,2020-09-08,3
91,2020-10-01,3
97,2020-10-14,2
87,2020-09-23,2
95,2020-10-09,2
38,2020-04-29,2
40,2020-05-14,2
43,2020-05-20,2


### How many times a week did this happen? 

In [26]:
(days2020["count"].sum() / 52).round(2)

3.19

---

### Just Carter cases

In [27]:
df["carter"] = df["narrative"].str.lower().str.contains("carter")

In [28]:
carter = df[df["carter"] == True]

In [29]:
carter.tail()

,id,date,time,shift,reporting_personnel,number_assisting,location,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,narrative,carter
403,202000040,2020-02-11,2205,NITE,"BETANCOURT, L",3.0,BOOKING AREA,2020,1,11,2,Tuesday,February,2020-02,2020-02-01,22,5,22:05:00,"At approximately 2210 prisoner Meehan,Brandon Lee was placed in the wrap after refusing to stand up and come out \nDOB\nof the holding cell. The prisoner became belligerent while also using profanity and pulling away when arresting Ofc Carter attempted to talk to the prisoner and assist the prisoner off the floor. I assisted Ofc Carter in getting the prisoner off the floor when the prisoner began pulling away from Ofc Carter. At this point Ofc Carter and my self placed the prisoner`s arms behind his back and requested the wrap system. We placed the prisoner in the wrap system with the help of PSO Walker and PSO Jalil. After being placed in cell 1708 Meehan began complaining of having high blood pressure problems, mental health problems such as bipolar disorder and being schizophrenic and then an hour and a half after being in the wrap the priosoner started to complain about having pains in his stomach due to a previous herina which he said he had already been having pains before coming to the jail. When i came back to the speak to the prisoner on a check shortly after the prisoner admitted that he just wanted to get out of jail and that he could sign a release for medical in order to get released. I explained to the prisoner that if he was seen by medical and needed to go to the hospital he would be released to go but would not be getting released to go home straight from the jail. When EMS did a vitals check on the prisoner they said everything looked normal and there was no need to worry especially about the blood pressure being too high. Meehan insisted in going because of pain in stomach due to the herina. Prisoner Meehan,Brandon was transported to the hospital and released from the jail with a rojo. [02/11/2020 01:38, LBETANCOURT, , PPD] \nPA",True
527,202000321,2020-11-11,1830,EVES,"MOODY, P",3.0,BOOKING AREA,2020,4,11,11,Wednesday,November,2020-11,2020-11-01,18,30,18:30:00,"At approximately 1830, inmate Jackson, Keenan was brought to jail by officer J. Carter. During the searching process \nDOB\ninmate began to not follow directions given by officer Carter and myself. Inmate then told jail staff if we took the handcuffs off to search him he would not co operate with jail staff orders and even attempt to fight jail staff. Officer Carter and myself attempted to reason with Mr. Jackson multiple times to co operate with our booking and searching procedures and to make his stay with us as easy as possible. Mr. Jackson continued to raise his voice at jail staff and promising that he would fight if he was removed from handcuffs. Jail staff then placed Mr. Jackson into the restraint chair with little resistance. No further conflict occurred. [11/11/2020 18:54, GGONZALEZ, , \nPAY \nPPD]",True
556,202100012,2021-01-19,1915,EVES,"OROZCO, C",2.0,SALLY PORT,2021,1,19,1,Tuesday,January,2021-01,2021-01-01,19,15,19:15:00,"Inmate was placed into restraint chair upon arrival by myself, PSO Cortes and Ofc Carter. Due to threatening to kill officers while en route to the jail and making threats to harm staff [01/19/2021 20:34, COROZCO, , PPD] \nPA",True
585,202100074,2021-03-15,1804,EVES,"KIDD, J",2.0,CELL,2021,1,15,3,Monday,March,2021-03,2021-03-01,18,4,18:04:00,"Inmate pushed button notifying jailers that he was suicidal. Inmate was then placed into the restraint chair by PSO Kidd and PSO Teel. Inmate was put into chair with no further incident and Sgt Hibbs was notified. [03/15/2021 18:05, JKIDD, , PPD] After inmate was \nPAY \nin chair for approximately 10 minutes PSO Teel made the decision to move the inmate to the wrap. The inmate was then moved to the wrap by PSO Kidd, PSO Teel, and OFC Carter. [03/15/2021 18:23, JKIDD, , PPD] Inmate 

---

### Exports

In [30]:
carter.to_csv("data/processed/carter_restraint_cases.csv", index=False)

In [31]:
df.to_csv("data/processed/all_restraint_cases.csv", index=False)

In [32]:
chair_hours.save("visuals/chair_hours_frequency.png", scale_factor=1)

In [33]:
chair_months.save("visuals/chair_month_year_frequency.png", scale_factor=1)